In [2]:
from sympy import (
    solve,
    Poly,
    Eq,
    Function,
    exp,
    Indexed,
    IndexedBase,
    Tuple,
    sqrt,
    Symbol,
    sin,
    limit,
    root,
    real_root,
    S,
    nroots,
)
from sympy.solvers.solvers import unrad
import numpy as np
import pandas as pd
from scipy.optimize import fsolve
from scipy.optimize import minimize

In [3]:
N = 15  # pe rate range 285 - 350 by 5s
M = 20  # es_rate range 160 - 255 by 5s
O = 10  # pm_rate range 150 - 195 by 5s
P = 401  # hours index from 0.0 - 100.0 by 0.25s


In [4]:
pe_rate = pd.DataFrame([285 + (iter * 5) for iter in range(N)])  # init dataframe for pe_rate
pe_hours = pd.DataFrame(iter * 0.25 for iter in range(P))  # init dataframe for pe_hours
es_rate = [160 + (iter * 5) for iter in range(M)]  # init dataframe for es_rate
es_hours = [iter * 0.25 for iter in range(P)]  # init dataframe for es_hours
pm_rate = [150 + (iter * 5) for iter in range(O)]  # init dataframe for pm_rate
pm_hours = [iter * 0.25 for iter in range(P)]  # init dataframe for pm_hours


In [5]:
df_pe = pd.DataFrame((np.outer(pe_hours, pe_rate))) # init pe df, labels
df_pe.columns = [285 + (iter * 5) for iter in range(N)]
df_pe.index = [iter * 0.25 for iter in range(P)]


In [6]:
df_es = pd.DataFrame((np.outer(es_hours, es_rate))) # init es df, labels
df_es.columns = [160 + (iter * 5) for iter in range(M)]
df_es.index = [iter * 0.25 for iter in range(P)]


In [7]:
df_pm = pd.DataFrame((np.outer(pm_hours, pm_rate))) # init pm df, labels
df_pm.columns = [160 + (iter * 5) for iter in range(O)]
df_pm.index = [iter * 0.25 for iter in range(P)]


In [8]:
search = 127.5

In [9]:
pm_mask = df_pm.mask(df_pm != search, False) # 

In [10]:
df_pm = pm_mask[pm_mask.any(axis=1)]

In [11]:
df_pm

,160,165,170,175,180,185,190,195,200,205
0.75,False,False,False,False,127.5,False,False,False,False,False


In [12]:
pm_id = df_pm.loc[:, (df_pm != 0).any(axis=0)]

In [13]:
pos = [0]
pm_colname = df_pm.columns[pos]
pm_indexname = df_pm.index[pos]

print(
    pm_indexname.tolist(), "hours at", pm_colname.tolist(), "per hour."
)


[0.75] hours at [160] per hour.


In [14]:

def breakdown(bgt):
    y = bgt - pm_id.values
    if y >= 0:
        return y
    else:
        return y


In [15]:
breakdown(300)

array([[172.5]], dtype=object)